In [ ]:
import json
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import rankfm
from lightfm import LightFM

In [ ]:
num_interactions = 1000000
interactions_df= pd.read_csv("interactions.csv",nrows=num_interactions)
interactions_df.head(100)

In [ ]:
interactions = interactions_df.values.tolist()
del interactions_df

In [ ]:
print(interactions[:10])

In [ ]:
frequencies = defaultdict(int)
songsPerPlaylist = defaultdict(list)
playlistsPerSong = defaultdict(list)
songIds = {}
playlistIds = set()
for row in tqdm(interactions):
    
    song = row[2]
    songIds[song] = len(songIds) #Assiging ID to song
    song = songIds[song]  
    frequencies[song] += 1
    pid = row[1]
    songsPerPlaylist[pid].append(song)
    playlistsPerSong[song].append(pid)

In [ ]:
nPlaylists,nSongs = len(songsPerPlaylist.keys()),len(songIds)
print(nPlaylists,nSongs)

In [ ]:
interactions_fm = [[d[1],songIds[d[2]]] for d in interactions]
n = len(interactions_fm)
print(n)
print(interactions_fm[:100])

### LightFM

In [ ]:
model = LightFM(loss='warp')

In [ ]:
### load coo_matrix from Scipy.sparse module
from scipy.sparse import coo_matrix
random.shuffle(interactions_fm)  #Shuffling the interactions data
#print(interactions_fm[:10])
#Splitting the interactions data
n = len(interactions_fm)
interactions_train, interactions_test = interactions_fm[:9*n//10], interactions_fm[9*n//10:]
#print(interactions_test[:10])
print(len(interactions_train),len(interactions_test))
row_indices_train = [d[0] for d in interactions_train]
column_indices_train = [d[1] for d in interactions_train]
values_train = [1 for i in range(len(interactions_train))]

interactions_matrix_train = coo_matrix((values_train,(row_indices_train,column_indices_train)))

print(interactions_matrix_train.shape)

row_indices_test = [d[0] for d in interactions_test]
column_indices_test = [d[1] for d in interactions_test]
values_test = [1 for i in range(len(interactions_test))]

print(len(row_indices_test),len(column_indices_test),len(values_test))

interactions_matrix_test = coo_matrix((values_test,(row_indices_test,column_indices_test)))

print(interactions_matrix_test.shape)

In [ ]:
%time model.fit(interactions_matrix_train, epochs=50, num_threads=4, verbose=True)

In [ ]:
from lightfm.evaluation import precision_at_k, auc_score

%time print("Train precision: %.2f" % precision_at_k(model, interactions_matrix_train, k=5).mean())
%time print("Test precision: %.2f" % precision_at_k(model, interactions_matrix_test, k=5).mean())

In [ ]:
from lightfm.evaluation import auc_score
%time print("Train auc score: %.4f" % auc_score(model, interactions_matrix_train).mean())
%time print("Test suc score: %.4f" % auc_score(model, interactions_matrix_test).mean())